# Knowledge Graph Question Answering for Building Knowledge Graphs using a ReAct Framework

This notebook implements an agentic workflow based on the **ReAct (Reasoning and Acting)** framework to translate natural language questions into precise SPARQL queries. The core of this notebook is a multi-turn refinement process where two specialized agents collaborate to produce, critique, and improve a SPARQL query until it is deemed correct.

---

## 🤖 Methodology: The Two-Agent Loop

The agentic workflow is orchestrated by the `SparqlRefinementAgent` class and consists of an iterative loop between two Large Language Model (LLM) powered agents:

1.  **Query Writer Agent**:
    * **Role**: To generate and revise SPARQL queries.
    * **Action**: Given a natural language question and a subset (defined by `num_triples` of the graph) of the graph, it writes an initial SPARQL query. In subsequent turns, it revises the query based on feedback.

2.  **Critique Agent**:
    * **Role**: To evaluate the query's correctness.
    * **Action**: After the Writer's query is executed, the Critique agent reviews the original question, the query itself, and a summary of the execution results. It then makes a decision:
        * `FINAL`: The query is correct and successfully answers the question. The loop terminates.
        * `IMPROVE`: The query is incorrect, incomplete, or inefficient. The agent provides specific, actionable feedback.

This feedback is then passed back to the Query Writer Agent, which begins the next iteration by generating an improved query. This cycle continues until a `FINAL` decision is reached or the maximum number of iterations is exceeded.

---

## ⚙️ Execution Workflow

The notebook follows a systematic process for each building and question:

1.  **Configuration**: Configure your API keys and URL for LLM calls.
2.  **Initialize Agents**: The `SparqlRefinementAgent` is instantiated.
3.  **Define a helper function for testing a single question**: `run_single_question` function is defined for initial testing.
4.  **Configure the necessary test conditions**: Key parameters such as the target building (`BUILDING_NAME`), LLM (`MODEL_NAME`), and SPARQL endpoint are set.
5.  **Process All Questions and Buildings**: The script iterates through each question in the JSON file.
    * The **ReAct loop** is triggered for the current question.
    * The final query generated by the agent is executed.
    * The ground-truth query is executed for comparison.
    * Key metrics are computed and saved. 

---

## 📊 Evaluation Metrics

To rigorously assess the correctness of the generated SPARQL query, the following metrics are calculated and logged:

* **Arity Matching F1**: Checks if the query returned the correct **number of columns**.
* **Exact Match F1**: A strict check for identical row content and column **order**, though it ignores column names.
* **Entity Set F1**: Flexibly checks if the correct **sets of values** were retrieved in each column, regardless of row structure or column order.
* **Row Matching F1**: The most robust metric. It finds the optimal column alignment and then checks for an exact, row-for-row match of the content. This is the primary indicator of a perfectly correct query.

In [1]:
# Standard library imports
import csv
import itertools
import json
import os
import re
import time
import traceback
import uuid
import warnings
from typing import Any, Dict, List, Optional

# Third-party imports
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
from pydantic import BaseModel, Field, ValidationError
from pyparsing import ParseException
from rdflib import BNode, Graph, Literal, URIRef
from SPARQLWrapper import JSON, SPARQLWrapper
from ReAct_agent.utils import get_kg_subset_content, extract_prefixes_from_ttl, check_if_question_exists, CsvLogger

from ReAct_agent.sparql_refinement_agent import SparqlRefinementAgent 



### 1. Assign your API key and your base url

In [ ]:

os.environ['OPENAI_API_KEY'] = "https://www.youtube.com/watch?v=dQw4w9WgXcQ" # Change this to your actual API key for whatever service you are using or set it in your environment variables
client = OpenAI(    
    api_key=os.environ.get('OPENAI_API_KEY'),
    base_url="https://api.openai.com/v1/"
)


### 2. Initialize The `SparqlRefinementAgent` (for testing)

In [ ]:

MODEL_NAME = "openai/o3-mini"
BUILDING_NAME = "bldg11"
SPARQL_TARGET = f"http://Ozans-MacBook-Pro-9.local:7200/repositories/{BUILDING_NAME}" 

LOG_FIELDNAMES = [
    'query_id', 'question_number', 'source', 'question', 'model',
    'ground_truth_sparql', 'generated_sparql',
    'syntax_ok', 'returns_results', 'perfect_match',
    'gt_num_rows', 'gt_num_cols',
    'gen_num_rows', 'gen_num_cols',
    'arity_matching_f1',
    'exact_match_f1',
    'entity_set_f1',
    'row_matching_f1',
    'less_columns_flag',
    'prompt_tokens', 'completion_tokens', 'total_tokens'
]


agent = SparqlRefinementAgent(
    sparql_endpoint=SPARQL_TARGET, 
    model_name=MODEL_NAME, 
    max_iterations=3,
    client=client
)

🌐 Remote SPARQL endpoint mode activated: http://Ozans-MacBook-Pro-9.local:7200/repositories/bldg11


### 3. Define a helper function for testing a single question

In [ ]:

def run_single_question():
    # The script will automatically detect if the target is a file path or a URL
    SPARQL_TARGET = LOCAL_TTL_PATH if USE_LOCAL_TTL_FILE else REMOTE_ENDPOINT_URL

    BRICK_PREFIXES = extract_prefixes_from_ttl(LOCAL_TTL_PATH)

    KNOWLEDGE_GRAPH_CONTENT = get_kg_subset_content(LOCAL_TTL_PATH, max_triples= num_triples)
    print("First 1000 chars of knowledge graph content:")
    print(KNOWLEDGE_GRAPH_CONTENT[:1000])
    with open(json_file_path, 'r', encoding='utf-8') as f:
        all_data = json.load(f)
    target_building_data = all_data[0]

    if not os.path.exists(os.path.dirname(LOG_FILE)):
        os.makedirs(os.path.dirname(LOG_FILE))

    logger = CsvLogger(filename=LOG_FILE, fieldnames=LOG_FIELDNAMES)
    agent = SparqlRefinementAgent(
        sparql_endpoint=SPARQL_TARGET, 
        model_name=MODEL_NAME, 
        max_iterations=3,
        client=client
    )

    print(f"--- Processing building: {BUILDING_NAME} for model: {MODEL_NAME} ---")

    first_question_processed = False

    try:
        for query_info in target_building_data.get('queries', []):
            query_id = query_info.get('query_id')
            ground_truth_sparql = query_info.get('sparql_query')

            if not ground_truth_sparql:
                print(f"⚠️ Warning: Skipping Query Group ID {query_id} (no ground truth SPARQL).")
                continue
                
            if "prefix" not in ground_truth_sparql.lower():
                ground_truth_sparql = BRICK_PREFIXES + ground_truth_sparql

            print(f"\n--- Processing Query Group ID: {query_id} ---")
            
            for question_obj in query_info.get('questions', []):
                question_text = question_obj.get('text')
                if not question_text:
                    continue

                if check_if_question_exists(question_text, LOG_FILE, MODEL_NAME):
                    print(f"Question '{question_text[:50]}...' already logged. Skipping.")
                    continue

                eval_data = {
                    'query_id': query_id,
                    'question_number': question_obj.get('question_number', 'N/A'),
                    'source': question_obj.get('source', 'N/A'),
                    'question': question_text,
                    'ground_truth_sparql': ground_truth_sparql
                }
                
                agent.refine_and_evaluate_query(
                    eval_data=eval_data, 
                    logger=logger, 
                    prefixes=BRICK_PREFIXES,
                    knowledge_graph_content=KNOWLEDGE_GRAPH_CONTENT
                )
                
                # --- Set flag and break after the first processed question ---
                print("\nFirst question processed. Exiting loops.")
                first_question_processed = True
                break # Exit the inner (questions) loop
            # --- Check the flag to exit the outer loop as well ---
            if first_question_processed:
                break # Exit the outer (queries) loop

    finally:
        logger.close()
        print("Logger closed.")

###  4.Configure the necessary test conditions: 
Set key parameters such as the target building (`BUILDING_NAME`), LLM (`MODEL_NAME`), and SPARQL endpoint.


In [10]:

MODEL_NAME = "openai/o3-mini"
BUILDING_NAME = "bldg11"
num_triples = 100
USE_LOCAL_TTL_FILE = False # We are using GraphDB for running queries. Convert this to True if you want to run queries locally.
LOCAL_TTL_PATH = f"./eval_buildings/{BUILDING_NAME}.ttl"
REMOTE_ENDPOINT_URL = f"http://Ozans-MacBook-Pro-9.local:7200/repositories/{BUILDING_NAME}" 
json_file_path = f"./Benchmark_QA_pairs/{BUILDING_NAME}_combined.json"
LOG_FILE = f"Example_Results/ReAct(w{num_triples})_{BUILDING_NAME}.csv"

run_single_question()


✅ Successfully extracted 23 prefixes from bldg11.ttl.
🔎 Original graph 'bldg11.ttl' contains 62578 triples.
   -> Graph is larger than 100 triples. Creating a subset for the prompt...
   -> ✅ Successfully created subset context with 100 triples.
First 1000 chars of knowledge graph content:
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix bsh: <https://brickschema.org/schema/BrickShape#> .
@prefix ns2: <http://buildsys.org/ontologies/bldg11#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix rec: <https://w3id.org/rec#> .
@prefix ref: <https://brickschema.org/schema/Brick/ref#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix tag: <https://brickschema.org/schema/BrickTag#> .
@prefix unit: <http://qudt.org/vocab/unit/> .
@prefix xsd: <http://ww

KeyboardInterrupt: 

### 5. Process All Questions and Buildings

In [ ]:
def run_all_buildings(model_name: str, num_triples: int):
    # --- Configure your details here ---
    for BUILDING_NAME in ["b59", "bldg11","TUC_building","dflexlibs_multizone"]:  
        MODEL_NAME = model_name
        # The script will automatically detect if the target is a file path or a URL
        SPARQL_TARGET = LOCAL_TTL_PATH if USE_LOCAL_TTL_FILE else REMOTE_ENDPOINT_URL

        LOG_FILE = f"Results/ReAct(w{num_triples})_{BUILDING_NAME}.csv"

        # --- Dynamically get prefixes from the building's TTL file ---
        # This now correctly points to the local ttl file regardless of the query target.
        BRICK_PREFIXES = extract_prefixes_from_ttl(LOCAL_TTL_PATH)
        if not BRICK_PREFIXES:
            print("Could not extract prefixes. Exiting.")
            exit()
        
        KNOWLEDGE_GRAPH_CONTENT = get_kg_subset_content(LOCAL_TTL_PATH, max_triples= num_triples)
        print("First 1000 chars of knowledge graph content:")
        print(KNOWLEDGE_GRAPH_CONTENT[:1000])

        
        try:
            with open(json_file_path, 'r', encoding='utf-8') as f:
                all_data = json.load(f)
        except FileNotFoundError:
            print(f"❌ Error: The file '{json_file_path}' was not found.")
            exit()
        except json.JSONDecodeError:
            print(f"❌ Error: The file '{json_file_path}' is not a valid JSON file.")
            exit()

        if not isinstance(all_data, list) or not all_data:
            print(f"❌ Error: Expected JSON to be a non-empty list of building objects.")
            exit()
        target_building_data = all_data[0]

        #if log file dont exist, mkdir
        if not os.path.exists(os.path.dirname(LOG_FILE)):
            os.makedirs(os.path.dirname(LOG_FILE))


        logger = CsvLogger(filename=LOG_FILE, fieldnames=LOG_FIELDNAMES)
        agent = SparqlRefinementAgent(
            sparql_endpoint=SPARQL_TARGET, 
            model_name=MODEL_NAME, 
            max_iterations=3
        )

        print(f"--- Processing building: {BUILDING_NAME} for model: {MODEL_NAME} ---")
        
        try:
            for query_info in target_building_data.get('queries', []):
                query_id = query_info.get('query_id')
                ground_truth_sparql = query_info.get('sparql_query')

                if not ground_truth_sparql:
                    print(f"⚠️ Warning: Skipping Query Group ID {query_id} (no ground truth SPARQL).")
                    continue
                    
                if "prefix" not in ground_truth_sparql.lower():
                    ground_truth_sparql = BRICK_PREFIXES + ground_truth_sparql

                print(f"\n--- Processing Query Group ID: {query_id} ---")
                
                for question_obj in query_info.get('questions', []):
                    question_text = question_obj.get('text')
                    if not question_text:
                        continue

                    if check_if_question_exists(question_text, LOG_FILE, MODEL_NAME):
                        continue

                    eval_data = {
                        'query_id': query_id,
                        'question_number': question_obj.get('question_number', 'N/A'),
                        'source': question_obj.get('source', 'N/A'),
                        'question': question_text,
                        'ground_truth_sparql': ground_truth_sparql
                    }
                    
                    agent.refine_and_evaluate_query(
                        eval_data=eval_data, 
                        logger=logger, 
                        prefixes=BRICK_PREFIXES,
                        knowledge_graph_content=KNOWLEDGE_GRAPH_CONTENT
                    )
        
        finally:
            logger.close()


for num_triples in [100, 5000]:
    run_all_buildings(model_name="openai/o3-mini", num_triples=num_triples)